In [1]:
import anndata
import numpy as np
import scipy as sp
import sys
# import pandas as pd
import torch
import torch_geometric
import os.path
sys.path.insert(1, '../')
import matplotlib.pyplot as plt
from scipy.io import mmread, mmwrite
from numpy import savetxt
%load_ext autoreload
%autoreload 2

In [2]:
import topovelo as tpv

In [3]:
dataset = 'Pancreas'
root = '/scratch/blaauw_root/blaauw1/gyichen'
adata = anndata.read_h5ad(f'{root}/data/{dataset}_pp.h5ad')

In [4]:

model_path_base = f'{root}/checkpoints/{dataset}'
figure_path_base = f'{root}/figures/{dataset}'
data_path = f'{root}/data/velovae/continuous/{dataset}'
gene_plot = tpv.sample_genes(adata, 4, 'means')

In [47]:
figure_path = f'{figure_path_base}/Graph'
model_path = f'{model_path_base}/Graph'

torch.manual_seed(2022)
np.random.seed(2022)

vae = tpv.VAE(adata, 
              tmax=20, 
              dim_z=5, 
              device='cuda:0',
              attention=False)

Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Detected 901 velocity genes.
Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

3 clusters detected based on gene co-expression.
(0.49, 0.34843595768045504), (0.51, 0.7329538970887918)
(0.55, 0.31774595674199757), (0.45, 0.7678039422838004)
(0.57, 0.812504473951968), (0.43, 0.44401169136605384)
KS-test result: [0. 0. 0.]
Initial induction: 1163, repression: 837/2000


In [6]:
graph = adata.obsp['connectivities']

In [48]:
config = {
    'early_stop_thred': 0.5
}
vae.train(adata, graph, config=config, plot=True, gene_plot=gene_plot, figure_path=figure_path)
#vae.save_model(model_path, 'encoder', 'decoder')
#vae.save_anndata(adata, 'velovae', data_path, file_name=f'{dataset}.h5ad')

Learning Rate based on Data Sparsity: 0.0002
--------------------------- Train a VeloVAE ---------------------------
*********               Creating a Graph Dataset              *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Epoch 1: Train ELBO = -20669.241,	Test ELBO = -41598.500,	Total Time =   0 h :  0 m :  4 s
Epoch 100: Train ELBO = 1386.257,	Test ELBO = 1641.096,	Total Time =   0 h :  2 m : 44 s
Epoch 200: Train ELBO = 1658.981,	Test ELBO = 1933.490,	Total Time =   0 h :  5 m : 24 s
Epoch 300: Train ELBO = 1712.324,	Test ELBO = 1824.349,	Total Time =   0 h :  8 m :  5 s
Epoch 400: Train ELBO = 1841.306,	Test ELBO = 1968.972,	Total Time =   0 h : 10 m : 45 s
Epoc

  0%|          | 0/3696 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.038
Average Set Size: 74
Epoch 621: Train ELBO = -894.095,	Test ELBO = -2970.115,	Total Time =   0 h : 16 m : 44 s
Epoch 700: Train ELBO = 5105.028,	Test ELBO = 5128.984,	Total Time =   0 h : 17 m : 44 s
Epoch 800: Train ELBO = 5441.212,	Test ELBO = 5516.784,	Total Time =   0 h : 18 m : 59 s
Epoch 900: Train ELBO = 5689.731,	Test ELBO = 5764.171,	Total Time =   0 h : 20 m : 13 s
Epoch 1000: Train ELBO = 5883.745,	Test ELBO = 5948.661,	Total Time =   0 h : 21 m : 27 s
Epoch 1100: Train ELBO = 6052.440,	Test ELBO = 6082.863,	Total Time =   0 h : 22 m : 42 s
Change in noise variance: 0.5354
*********             Velocity Refinement Round 2              *********
Epoch 1121: Train ELBO = 5774.401,	Test ELBO = 5833.003,	Total Time =   0 h : 23 m :  2 s
Epoch 1200: Train ELBO = 5937.431,	Test ELBO = 5964.156,	Total Time =   0 h : 24 m :  2 s
Epoch 1300: Train ELBO = 6047.879,	Test ELBO = 6070.212,	Total Time =   0 h : 25 m : 16 s
Epoch 1400: Train ELBO = 6130.16

In [33]:
vae.save_anndata(adata, 'velovae', data_path, file_name=None)

In [39]:
cluster_edges = [('Ngn3 low EP', 'Ngn3 high EP'),
                 ('Ngn3 high EP', 'Pre-endocrine'),
                 ('Pre-endocrine', 'Delta'),
                 ('Pre-endocrine', 'Beta'),
                 ('Pre-endocrine', 'Epsilon'),
                 ('Pre-endocrine', 'Alpha')]
res, res_type = tpv.post_analysis(adata,
                                 dataset,
                                 ['VeloVAE'],
                                 ['velovae'],
                                 compute_metrics=True,
                                 genes=gene_plot,
                                 grid_size=(1,4),
                                 figure_path=figure_path,
                                 cluster_edges=cluster_edges)

Computing velocity embedding using scVelo
computing velocity graph (using 1/32 cores)


  0%|          | 0/3696 [00:00<?, ?cells/s]

    finished (0:00:29) --> added 
    'velovae_velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:00) --> added
    'velovae_velocity_umap', embedded velocity vectors (adata.obsm)
---     Computing Peformance Metrics     ---
Dataset Size: 3696 cells, 2000 genes
---   Plotting  Results   ---
saving figure to file /scratch/blaauw_root/blaauw1/gyichen/figures/Pancreas/Graph/Pancreas_velovae_stream.png
